In [1]:
using QuantumCumulants
using Symbolics
using SymbolicUtils
using Formatting

In [2]:
# Parameters
N = 4 #number of atoms
M = 2
kappa, n, Δ, ilist, qlist, gamma_par, etap, gamma_perp, eta =
    cnumbers("κ ñ Δ_c I(t) Q(t) γ_{∥} η_p γ_{⊥} η")

wf, G, kappa_f = cnumbers("Δ_f G κ_f")


g(j) = cnumbers(Symbol("g_{$j}"))[1]
Δs(j) = cnumbers(Symbol("Δ_{$j}"))[1]

# Hilbertspace
hf = FockSpace(:cavity)
hfilter = FockSpace(:cavity)
ha = ⊗([NLevelSpace(Symbol(:atom, i), 2) for i = 1:N]...)
h = hf ⊗ ha

ℋ(cavity) ⊗ ℋ(atom1) ⊗ ℋ(atom2) ⊗ ℋ(atom3) ⊗ ℋ(atom4)

In [3]:
# Operators
a = Destroy(h, :a, 1)
#b = Destroy(h, :b, 7)
sigma(i, j, k) = Transition(h, Symbol("σ_{$k}"), i, j, k + 1)
sigmaz(k) = sigma(2, 2, k) - sigma(1, 1, k)
sigmap(k) = sigma(2, 1, k)
sigmam(k) = sigma(1, 2, k)

sigmam (generic function with 1 method)

In [4]:
H = Δ * a'a + 1im * ilist * (a' - a) + qlist * (a' + a)
for j = 1:N
    H += Δs(j) / 2 * sigmaz(j) + 1im * g(j) * (a' * sigmam(j) - a * sigmap(j))
end
#filter resonator
#H += wf*b'b+G*(a'b+b'a)

In [5]:
# Collapse operators

rates = Any[2*(n+1)*kappa, 2*n*kappa]
rates = append!(rates, [gamma_par * (n + 1) for i = 1:N])
rates = append!(rates, [n * gamma_par + etap for i = 1:N])
rates = append!(rates, [gamma_perp / 2 for i = 1:N])
#filter resonator
#rates = append!(rates, [2*kappa_f])


J = Any[a, a']
J = append!(J, [sigmam(i) for i = 1:N])
J = append!(J, [sigmap(i) for i = 1:N])
J = append!(J, [sigmaz(i) for i = 1:N])
#filter resonator
#J = append!(J, [b]);

In [7]:
subdict = Dict([])
subdict1 = Dict([])
subdict2 = Dict([])
subdict3 = Dict([])
vec = [
    "\u2081",
    "\u2082",
    "\u2083",
    "\u2084",
    "\u2085",
    "\u2086",
    "\u2087",
    "\u2088",
    "\u2089",
]
ops = Any[a, a'*a, a*a]

append!(ops, [a*sigmam(i) for i = 1:N])
append!(ops, [a'*sigmam(i) for i = 1:N])
append!(ops, [a*sigmaz(i) for i = 1:N])
append!(ops, [sigmam(i) for i = 1:N])
append!(ops, [sigmaz(i) for i = 1:N])
append!(ops, [sigmam(j)*sigmam(i) for i = 1:N for j=1:N if (i!=j)])
append!(ops, [sigmam(j)*sigmap(i) for i = 1:N for j=1:N if (i!=j)])
append!(ops, [sigmaz(j)*sigmam(i) for i = 1:N for j=1:N if (i!=j)])
append!(ops, [sigmaz(j)*sigmaz(i) for i = 1:N for j=1:N if (i!=j)])

for i = 1:N
    subs = vec[i]
    sm = "σ{$subs}⁻"
    sp = "σ{$subs}⁺"
    sz = "σ{$subs}ᶻ"
    subdict1 = merge!(subdict1, Dict([average(a) => 0]))
    subdict1 = merge!(subdict1, Dict([average(a') => 0]))
    #subdict1 = merge!(subdict1, Dict([average(b)=>0]))
    #subdict1 = merge!(subdict1, Dict([average(b')=>0]))
    subdict1 = merge!(subdict1, Dict([average(sigmam(i)) => 0]))
    subdict1 = merge!(subdict1, Dict([average(sigmap(i)) => 0]))
    #subdict1 = merge!(subdict1, Dict([(i)=>0]))
    #subdict1 = merge!(subdict1, Dict([(q)=>Symbol("0")]))

    subdict = merge!(
        subdict,
        Dict([average(a * sigmam(i)) => Parameter(Symbol("⟨a" * sm * "⟩"))]),
    )
    subdict = merge!(
        subdict,
        Dict([average(a' * sigmam(i)) => Parameter(Symbol("⟨a^†" * sm * "⟩"))]),
    )

    subdict = merge!(
        subdict,
        Dict([average(a * sigmap(i)) => Parameter(Symbol("⟨a" * sp * "⟩"))]),
    )
    subdict = merge!(
        subdict,
        Dict([average(a' * sigmap(i)) => Parameter(Symbol("⟨a^†" * sp * "⟩"))]),
    )

    subdict = merge!(
        subdict,
        Dict([
            average(a * sigma(2, 2, i)) =>
                Parameter(Symbol("⟨a" * sz * "⟩")) / 2 + average(a) / 2,
        ]),
    )
    subdict = merge!(
        subdict,
        Dict([
            average(a' * sigma(2, 2, i)) =>
                Parameter(Symbol("⟨a^†" * sz * "⟩")) / 2 + average(a') / 2,
        ]),
    )


    #subdict = merge!(subdict, Dict([average(b*sigmam(i))=>Parameter(Symbol("⟨b"*sm*"⟩"))]))
    #subdict = merge!(subdict, Dict([average(b'*sigmam(i))=>Parameter(Symbol("⟨b^†"*sm*"⟩"))]))

    #subdict = merge!(subdict, Dict([average(b*sigmap(i))=>Parameter(Symbol("⟨b"*sp*"⟩"))]))
    #subdict = merge!(subdict, Dict([average(b'*sigmap(i))=>Parameter(Symbol("⟨b^†"*sp*"⟩"))]))

    #subdict = merge!(subdict, Dict([average(b*sigma(2,2,i))=>Parameter(Symbol("⟨b"*sz*"⟩"))/2+average(b)/2]))
    #subdict = merge!(subdict, Dict([average(b'*sigma(2,2,i))=>Parameter(Symbol("⟨b^†"*sz*"⟩"))/2+average(b')/2]))


    for k = 1:N
        subsk = vec[k]
        smk = "σ{$subsk}⁻"
        spk = "σ{$subsk}⁺"
        szk = "σ{$subsk}ᶻ"

        subdict = merge!(
            subdict,
            Dict([
                average(sigmam(k) * sigmap(i)) =>
                    Parameter(Symbol("⟨" * smk * sp * "⟩")),
            ]),
        )
        subdict = merge!(
            subdict,
            Dict([
                average(sigmap(k) * sigmap(i)) =>
                    Parameter(Symbol("⟨" * spk * sp * "⟩")),
            ]),
        )
        subdict = merge!(
            subdict,
            Dict([
                average(sigmap(k) * sigmam(i)) =>
                    Parameter(Symbol("⟨" * spk * sm * "⟩")),
            ]),
        )
        subdict = merge!(
            subdict,
            Dict([
                average(sigmam(k) * sigmam(i)) =>
                    Parameter(Symbol("⟨" * smk * sm * "⟩")),
            ]),
        )

        subdict = merge!(
            subdict,
            Dict([
                average(sigma(2, 2, k) * sigmap(i)) =>
                    Parameter(Symbol("⟨" * szk * sp * "⟩")) / 2 +
                    Parameter(Symbol("⟨" * sp * "⟩")) / 2,
            ]),
        )
        subdict = merge!(
            subdict,
            Dict([
                average(sigma(2, 2, k) * sigmam(i)) =>
                    Parameter(Symbol("⟨" * szk * sm * "⟩")) / 2 +
                    Parameter(Symbol("⟨" * sm * "⟩")) / 2,
            ]),
        )

        subdict = merge!(
            subdict,
            Dict([
                average(sigmap(k) * sigma(2, 2, i)) =>
                    Parameter(Symbol("⟨" * spk * sz * "⟩")) / 2 +
                    Parameter(Symbol("⟨" * spk * "⟩")) / 2,
            ]),
        )
        subdict = merge!(
            subdict,
            Dict([
                average(sigmam(k) * sigma(2, 2, i)) =>
                    Parameter(Symbol("⟨" * smk * sz * "⟩")) / 2 +
                    Parameter(Symbol("⟨" * smk * "⟩")) / 2,
            ]),
        )

        subdict = merge!(
            subdict,
            Dict([
                average(sigma(2, 2, k) * sigma(2, 2, i)) =>
                    1 / 4 * (
                        Parameter(Symbol("⟨" * szk * sz * "⟩")) +
                        Parameter(Symbol("⟨" * sz * "⟩")) +
                        Parameter(Symbol("⟨" * szk * "⟩")) +
                        1
                    ),
            ]),
        )
    end

    subdict =
        merge!(subdict, Dict([average(sigmam(i)) => Parameter(Symbol("⟨" * sm * "⟩"))]))
    subdict =
        merge!(subdict, Dict([average(sigmap(i)) => Parameter(Symbol("⟨" * sp * "⟩"))]))
    subdict = merge!(
        subdict,
        Dict([average(sigma(2, 2, i)) => (Parameter(Symbol("⟨" * sz * "⟩")) / 2) + 1 / 2]),
    )


    subdict2 = merge!(subdict, Dict([etap => 0]))
    subdict2 = merge!(subdict, Dict([n => 0]))
    #subdict2 = merge!(subdict, Dict([qlist => 0]))
    #subdict2 = merge!(subdict, Dict([ilist => eta]))
    
    #subdict3 = merge!(subdict, Dict([average(a) => Parameter(Symbol("x_0"))+1im*Parameter(Symbol("x_1"))]))
    #subdict3 = merge!(subdict, Dict([average(a') => Parameter(Symbol("x_0"))-1im*Parameter(Symbol("x_1"))]))
    #subdict3 = merge!(subdict, Dict([average(a'a) => Parameter(Symbol("x_2"))+1im*Parameter(Symbol("x_3"))]))
    #subdict3 = merge!(subdict, Dict([average(a*a) => Parameter(Symbol("x_4"))+1im*Parameter(Symbol("x_5"))]))
    #subdict3 = [merge!(subdict, Dict([average(a*sigmam(k)) => Parameter(Symbol("x_"*string(N*k+5-1)))+1im*Parameter(Symbol("x_"*string(N*k+6-1)))])) for k=1:(N)]
    #subdict3 = [merge!(subdict, Dict([average(a'*sigmam(k)) => Parameter(Symbol("x_"*string(N*k+4*N)))+1im*Parameter(Symbol("x_"*string(N*k+5*N-1)))])) for k=1:(N)]
    #subdict3 = [merge!(subdict, Dict([average(a*sigmap(k)) => Parameter(Symbol("x_"*string(N*k+4*N)))-1im*Parameter(Symbol("x_"*string(N*k+5*N-1)))])) for k=1:(N)]
   # 
    #subdict3 = [merge!(subdict, Dict([average(a*sigmaz(k)) => Parameter(Symbol("x_"*string(N*k+4*N+N*2)))+1im*Parameter(Symbol("x_"*string(N*k+5*N-1+N*2)))])) for k=1:(N)]
    #subdict3 = [merge!(subdict, Dict([Parameter(Symbol("⟨a"* sz * "⟩")) => Parameter(Symbol("x_"*string(N*k+4*N+N*2)))+1im*Parameter(Symbol("x_"*string(N*k+5*N-1+N*2)))])) for k=1:(N)]#

    #subdict3 = [merge!(subdict, Dict([average(a'*sigmaz(k)) => Parameter(Symbol("x_"*string(N*k+4*N+N*2)))-1im*Parameter(Symbol("x_"*string(N*k+5*N-1+N*2)))])) for k=1:(N)]
    #subdict3 = [merge!(subdict, Dict([Parameter(Symbol("⟨a^†"* sz * "⟩")) => Parameter(Symbol("x_"*string(N*k+4*N+N*2)))-1im*Parameter(Symbol("x_"*string(N*k+5*N-1+N*2)))])) for k=1:(N)]

    
    #subdict3 = [merge!(subdict, Dict([average(sigmam(k)) => Parameter(Symbol("x_"*string(N*k+4*N+N*4)))+1im*Parameter(Symbol("x_"*string(N*k+5*N-1+N*4)))])) for k=1:(N)]
    #subdict3 = [merge!(subdict, Dict([Parameter(Symbol("⟨" * sm * "⟩")) => Parameter(Symbol("x_"*string(N*k+4*N+N*4)))+1im*Parameter(Symbol("x_"*string(N*k+5*N-1+N*4)))])) for k=1:(N)]

    #subdict3 = [merge!(subdict, Dict([average(sigmaz(k)) => Parameter(Symbol("x_"*string(N*k+4*N+N*6)))+1im*Parameter(Symbol("x_"*string(N*k+5*N-1+N*6)))])) for k=1:(N)]
    #subdict3 = [merge!(subdict, Dict([Parameter(Symbol("⟨" * sz * "⟩")) => Parameter(Symbol("x_"*string(N*k+4*N+N*6)))+1im*Parameter(Symbol("x_"*string(N*k+5*N-1+N*6)))])) for k=1:(N)]

    
    
end
eqns = meanfield(sigmaz(1)'sigmam(2), H, J; rates = rates, order = M, simplify = true)

#careful here, this applies the approximations from the paper
#eqns = SymbolicUtils.simplify(Symbolics.substitute(eqns, subdict1), expand=true)

eqns = SymbolicUtils.simplify(Symbolics.substitute(eqns, subdict2), expand = true)

eq_n = SymbolicUtils.simplify(Symbolics.substitute(eqns, subdict), expand = true)

∂ₜ(var"⟨σ{₁}ᶻσ{₂}⁻⟩") = (0.0 - 1.0im)*var"Δ_{2}"*var"⟨σ{₁}ᶻσ{₂}⁻⟩" + -var"γ_{∥}"*var"⟨σ{₂}⁻⟩" + -var"γ_{⊥}"*var"⟨σ{₁}ᶻσ{₂}⁻⟩" + (-1.5 + 0.0im)var"γ_{∥}"*var"⟨σ{₁}ᶻσ{₂}⁻⟩" + var"g_{2}"*var"⟨aσ{₂}ᶻ⟩"*var"⟨σ{₁}ᶻ⟩" + var"g_{2}"*var"⟨aσ{₁}ᶻ⟩"*var"⟨σ{₂}ᶻ⟩" + var"g_{2}"*var"⟨σ{₁}ᶻσ{₂}ᶻ⟩"*⟨a⟩ + (-2.0 + 0.0im)var"g_{1}"*var"⟨a^†σ{₂}⁻⟩"*var"⟨σ{₁}⁻⟩" + (-2.0 + 0.0im)var"g_{1}"*var"⟨a^†σ{₁}⁻⟩"*var"⟨σ{₂}⁻⟩" + (-2.0 + 0.0im)var"g_{1}"*var"⟨aσ{₁}⁺⟩"*var"⟨σ{₂}⁻⟩" + (-2.0 + 0.0im)var"g_{1}"*var"⟨aσ{₂}⁻⟩"*var"⟨σ{₁}⁺⟩" + (-2.0 + 0.0im)var"g_{1}"*var"⟨σ{₁}⁻σ{₂}⁻⟩"*⟨a′⟩ + (-2.0 + 0.0im)var"g_{1}"*var"⟨σ{₁}⁺σ{₂}⁻⟩"*⟨a⟩ + (4.0 + 0.0im)var"g_{1}"*var"⟨σ{₁}⁻⟩"*var"⟨σ{₂}⁻⟩"*⟨a′⟩ + (-2.0 + 0.0im)var"g_{2}"*var"⟨σ{₁}ᶻ⟩"*var"⟨σ{₂}ᶻ⟩"*⟨a⟩ + (4.0 + 0.0im)var"g_{1}"*var"⟨σ{₁}⁺⟩"*var"⟨σ{₂}⁻⟩"*⟨a⟩


In [18]:
IJulia.qtconsole()

Process(`/usr/local/bin/jupyter qtconsole --existing /home/andreas/.local/share/jupyter/runtime/kernel-951f119c-9304-4eeb-94bf-ea8513ba9cb5.json`, ProcessRunning)